# Import packages

In [1]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from typing import List
import requests
from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI

# Define web_search function

In [2]:
def web_search(web_query: str, num_results: int) -> list[str]:
    return [r["link"] for r in DuckDuckGoSearchAPIWrapper().results(web_query, num_results)]

In [3]:
result = web_search(web_query="How many titles did Michael Jordan win?", num_results=5)
print(result)

['https://en.wikipedia.org/wiki/List_of_career_achievements_by_Michael_Jordan', 'https://www.sportsconduct.com/blogs/michael-jordan-championship-rings/', 'https://www.sportingnews.com/us/nba/chicago-bulls/news/michael-jordan-three-peat-1990s-chicago-bulls-dynasty-blocked-nba-legends/96d40982a19398fccb8b2e2b', 'https://www.gameslearningsociety.org/how-many-game-winners-did-michael-jordan-have/', 'https://en.as.com/nba/how-long-did-it-take-michael-jordan-to-win-an-nba-championship-n/']
['https://en.wikipedia.org/wiki/List_of_career_achievements_by_Michael_Jordan', 'https://www.sportsconduct.com/blogs/michael-jordan-championship-rings/', 'https://www.sportingnews.com/us/nba/chicago-bulls/news/michael-jordan-three-peat-1990s-chicago-bulls-dynasty-blocked-nba-legends/96d40982a19398fccb8b2e2b', 'https://www.gameslearningsociety.org/how-many-game-winners-did-michael-jordan-have/', 'https://en.as.com/nba/how-long-did-it-take-michael-jordan-to-win-an-nba-championship-n/']


# Define web_scrape function

In [4]:
def web_scrape(url: str) -> str:
    try:
        response = requests.get(url)
 
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            page_text = soup.get_text(separator=" ", strip=True)
 
            return page_text
        else:
            return f"Could not retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Could not retrieve the webpage: {e}"

In [5]:
result = web_scrape('https://en.wikipedia.org/wiki/List_of_career_achievements_by_Michael_Jordan')
print(result)

List of career achievements by Michael Jordan - Wikipedia Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk Contents move to sidebar hide (Top) 1 College statistics Toggle College statistics subsection 1.1 Averages 1.2 Totals 2 NBA career statistics Toggle NBA career statistics subsection 2.1 Averages 2.2 Totals 2.3 Playoffs 3 Media rankings 4 Amateur achievements & awards Toggle Amateur achievements & awards subsection 4.1 High School 4.2 College 5 Professional achievements Toggle Professional achievements subsection 5.1 Hall of Fame Inductions 5.2 NBA achievements 5.3 United States National Team 6 Media awards & honors 7 Off-court achievements & award

# Import custom functions

In [6]:
from utilities import to_obj, get_llm
from prompts import (
    ASSISTANT_SELECTION_PROMPT_TEMPLATE,
    WEB_SEARCH_PROMPT_TEMPLATE,
    SUMMARY_PROMPT_TEMPLATE,
    RESEARCH_REPORT_PROMPT_TEMPLATE
)

# Constrain output, define question, and instantiate llm

In [7]:
NUM_SEARCH_QUERIES = 2
NUM_SEARCH_RESULTS_PER_QUERY = 3
RESULT_TEXT_MAX_CHARACTERS = 10000
question = 'What can I see and do in the Spanish town of Astorga?'
llm = get_llm()

/home/hefin/Documents/Projects/LangChain-In-Action/utilities.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  return ChatOpenAI(


# Generate web search

In [8]:
assistant_selection_prompt = ASSISTANT_SELECTION_PROMPT_TEMPLATE.format(user_question=question)
assistant_instructions = llm.invoke(assistant_selection_prompt)

In [9]:
assistant_instructions_dict = to_obj(assistant_instructions.content)
print(assistant_instructions_dict)

{'assistant_type': 'Tour guide assistant', 'assistant_instructions': 'You are a world-travelled AI tour guide assistant. Your main purpose is to draft engaging, insightful, unbiased, and well-structured travel reports on given locations, including history, attractions, and cultural insights.', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}
{'assistant_type': 'Tour guide assistant', 'assistant_instructions': 'You are a world-travelled AI tour guide assistant. Your main purpose is to draft engaging, insightful, unbiased, and well-structured travel reports on given locations, including history, attractions, and cultural insights.', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}


In [10]:
web_search_prompt = WEB_SEARCH_PROMPT_TEMPLATE.format(
    assistant_instructions=assistant_instructions_dict['assistant_instructions'],
    num_search_queries=NUM_SEARCH_QUERIES,
    user_question=assistant_instructions_dict['user_question']
)

web_search_queries = llm.invoke(web_search_prompt)
web_search_queries_list = to_obj(web_search_queries.content.replace('\n', ''))

In [11]:
print(web_search_queries_list)

[{'search_query': 'Astorga Spain attractions history cultural insights', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}, {'search_query': 'things to do in Astorga Spain travel guide', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}, {'search_query': 'Astorga Spain tourism highlights', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}]
[{'search_query': 'Astorga Spain attractions history cultural insights', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}, {'search_query': 'things to do in Astorga Spain travel guide', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}, {'search_query': 'Astorga Spain tourism highlights', 'user_question': 'What can I see and do in the Spanish town of Astorga?'}]


In [12]:
searches_and_result_urls = [{'result_urls': web_search(web_query=wq['search_query'], 
                                     num_results=NUM_SEARCH_RESULTS_PER_QUERY), 
                           'search_query': wq['search_query']} 
                           for wq in web_search_queries_list]

In [13]:
print(searches_and_result_urls)

[{'result_urls': ['https://turispanish.com/tourism/astorga-spain-tourism/', 'https://www.eyeonspain.com/blogs/bestofspain/23350/astorga---a-must-visit.aspx', 'https://unusedrooms.com/astorga-castile-and-leon-region-of-spain/'], 'search_query': 'Astorga Spain attractions history cultural insights'}, {'result_urls': ['https://wanderlog.com/list/geoCategory/1589429/top-things-to-do-and-attractions-in-astorga', 'https://worldfreetours.com/blog/astorga/explore-astorga-for-free/', 'https://www.lonelyplanet.com/articles/camino-de-santiago-activities-beyond-the-trails'], 'search_query': 'things to do in Astorga Spain travel guide'}, {'result_urls': ['https://turispanish.com/tourism/astorga-spain-tourism/', 'https://www.eyeonspain.com/blogs/bestofspain/23350/astorga---a-must-visit.aspx', 'https://touristplaces.guide/top-10-must-visit-tourist-places-in-astorga/'], 'search_query': 'Astorga Spain tourism highlights'}]
[{'result_urls': ['https://turispanish.com/tourism/astorga-spain-tourism/', 'htt

# Flatten results

In [14]:
search_query_and_result_url_list = []
for qr in searches_and_result_urls:
    search_query_and_result_url_list.extend([{'search_query': qr['search_query'], 
                                    'result_url': r
                                    } for r in qr['result_urls']])

In [15]:
print(search_query_and_result_url_list)

[{'search_query': 'Astorga Spain attractions history cultural insights', 'result_url': 'https://turispanish.com/tourism/astorga-spain-tourism/'}, {'search_query': 'Astorga Spain attractions history cultural insights', 'result_url': 'https://www.eyeonspain.com/blogs/bestofspain/23350/astorga---a-must-visit.aspx'}, {'search_query': 'Astorga Spain attractions history cultural insights', 'result_url': 'https://unusedrooms.com/astorga-castile-and-leon-region-of-spain/'}, {'search_query': 'things to do in Astorga Spain travel guide', 'result_url': 'https://wanderlog.com/list/geoCategory/1589429/top-things-to-do-and-attractions-in-astorga'}, {'search_query': 'things to do in Astorga Spain travel guide', 'result_url': 'https://worldfreetours.com/blog/astorga/explore-astorga-for-free/'}, {'search_query': 'things to do in Astorga Spain travel guide', 'result_url': 'https://www.lonelyplanet.com/articles/camino-de-santiago-activities-beyond-the-trails'}, {'search_query': 'Astorga Spain tourism hig

# Scraping the results

In [16]:
result_text_list = [ {'result_text': web_scrape(url=re['result_url'])[:RESULT_TEXT_MAX_CHARACTERS],
                     'result_url': re['result_url'],
                     'search_query': re['search_query']}
                   for re in search_query_and_result_url_list]

In [17]:
print(result_text_list)

[{'result_text': "▷ Discovering Astorga: A Gem of Spanish Tourism ✅ 【 2025 】 Saltar al contenido Home Travel Health Passport Restrictions Insurance About Eli Contact Menú de navegación Menú de navegación Home Travel Health Passport Restrictions Insurance About Eli Contact Discovering Astorga: A Gem of Spanish Tourism por Elisabeth Gonzalez Smith agosto 29, 2024 Nestled in the heart of Spain, Astorga is a captivating blend of history, culture, and stunning architecture that beckons travelers from around the globe. Known for its rich Roman heritage and striking modernist buildings, including the iconic Gaudí-designed palace, Astorga offers a unique glimpse into both its ancient past and vibrant present. With charming streets, delectable local cuisine, and a warm, welcoming atmosphere, this hidden gem in the region of León is quickly becoming a must-visit destination for those seeking an authentic Spanish experience. Dive into the allure of Astorga, where every corner tells a story waitin

# Summarising results

In [18]:
result_text_summary_list = []
for rt in result_text_list: 
    summary_prompt = SUMMARY_PROMPT_TEMPLATE.format(
        search_result_text=rt['result_text'], 
        search_query=rt['search_query'])
    
    text_summary = llm.invoke(summary_prompt)
 
    result_text_summary_list.append({'text_summary': text_summary,
                         'result_url': rt['result_url'],
                         'search_query': rt['search_query']})

# Generating the research report

In [19]:
stringified_summary_list = [f'Source URL: {sr["result_url"]}\nSummary: {sr["text_summary"]}' 
                            for sr in result_text_summary_list]

appended_result_summaries = '\n'.join(stringified_summary_list)
print(appended_result_summaries)

Source URL: https://turispanish.com/tourism/astorga-spain-tourism/
Summary: content="Astorga, Spain, is a captivating town known for its rich history, culture, and stunning architecture. Established as Asturica Augusta in 14 BC by Emperor Octavian, it served as an important administrative and military hub in Roman times. Visitors can explore well-preserved archaeological sites, including Roman ruins, basilicas, thermal baths, and ancient mansions along the Roman Route.\n\nKey attractions include the iconic Gaudí-designed Episcopal Palace, which adds modernist flair to the historical landscape, and the majestic cathedral that represents the region's spiritual heritage. The charming cobbled streets are lined with cafes and shops, providing opportunities to savor local cuisine and experience vibrant culture.\n\nAstorga is also a gateway to beautiful surrounding landscapes, making it a popular stop for pilgrims on the Camino de Santiago and a destination for hikers. The nearest airport is 

In [20]:
research_report_prompt = RESEARCH_REPORT_PROMPT_TEMPLATE.format(
    research_summary=appended_result_summaries,
    user_question=question
)
research_report = llm.invoke(research_report_prompt)

In [21]:
print(research_report)

content='# Exploring Astorga, Spain: A Comprehensive Guide to Attractions and Activities\n\nAstorga, a picturesque town located in the province of León in northwestern Spain, is steeped in rich history, culture, and stunning architecture. As a key stop along the Camino de Santiago and the Silver Route, Astorga offers visitors a unique blend of ancient Roman heritage, medieval charm, and modern artistic expression, making it a must-visit destination for travelers seeking an enriching experience.\n\n## 1. Historical Significance\n\n### 1.1 Origins and Roman Heritage\n\nAstorga was originally established as **Asturica Augusta** in 14 BC by Emperor Octavian. Its historical importance stems from its role as a strategic administrative and military hub during Roman times, primarily due to its location near gold deposits. Over two millennia, Astorga has preserved its historical essence, showcasing various archaeological sites that reflect its Roman past. Visitors can explore well-preserved Rom

In [22]:
from IPython.display import display, Markdown
display(Markdown(research_report.content))

# Exploring Astorga, Spain: A Comprehensive Guide to Attractions and Activities

Astorga, a picturesque town located in the province of León in northwestern Spain, is steeped in rich history, culture, and stunning architecture. As a key stop along the Camino de Santiago and the Silver Route, Astorga offers visitors a unique blend of ancient Roman heritage, medieval charm, and modern artistic expression, making it a must-visit destination for travelers seeking an enriching experience.

## 1. Historical Significance

### 1.1 Origins and Roman Heritage

Astorga was originally established as **Asturica Augusta** in 14 BC by Emperor Octavian. Its historical importance stems from its role as a strategic administrative and military hub during Roman times, primarily due to its location near gold deposits. Over two millennia, Astorga has preserved its historical essence, showcasing various archaeological sites that reflect its Roman past. Visitors can explore well-preserved Roman ruins, including a forum, thermal baths, and ancient residences known as "domus." The **Roman Museum**, situated in the historic Ergastula (prison), displays artifacts such as statues, amphorae, jewelry, and reliefs, offering a glimpse into the daily lives of Astorga’s ancient inhabitants (Eye on Spain, n.d.).

### 1.2 The Convergence of Cultural Routes

Astorga serves as a convergence point for two significant cultural routes: the **Way of Saint James** (Camino de Santiago) and the **Silver Route**. The Camino de Santiago is a historic pilgrimage route that attracts thousands of travelers each year, while the Silver Route connects Astorga to the ancient Roman gold mining towns, enhancing its cultural significance (Eye on Spain, n.d.).

## 2. Architectural Marvels

Astorga's architectural landscape is a testament to its historical evolution, featuring a harmonious blend of styles from various periods.

### 2.1 Episcopal Palace

One of the town's most iconic landmarks is the **Episcopal Palace**, designed by the renowned architect **Antoni Gaudí** between 1889 and 1915. This neo-Gothic masterpiece resembles a fairy-tale castle with its striking towers and intricate details. The palace houses the **Los Caminos Museum**, which showcases items related to the Camino de Santiago, making it a vital cultural hub for visitors (Eye on Spain, n.d.).

### 2.2 Cathedral of Santa María

The **Cathedral of Santa María de Astorga** stands as a prominent example of the town's architectural grandeur. This Roman Catholic cathedral is a blend of **Gothic**, **Baroque**, and **Renaissance** styles, with construction spanning from the 15th to the 18th century. Notable features include its two towers, which are of different colors due to an earthquake in 1775, and the exquisite main altarpiece (Eye on Spain, n.d.; Lonely Planet, n.d.).

### 2.3 Plaza Mayor

The **Plaza Mayor** is the heart of Astorga, showcasing stunning neoclassical architecture, including the **Baroque Town Hall**. This vibrant square hosts a traditional street market every Tuesday morning, offering visitors a chance to mingle with locals and experience the town's lively atmosphere (Eye on Spain, n.d.).

## 3. Cultural Experiences

Astorga is not only a feast for the eyes but also a hub of cultural experiences that reflect its rich traditions.

### 3.1 Local Cuisine

Astorga is celebrated for its culinary delights, particularly its **chocolate**. The town's famous **chocolate con churros**, **cocido maragato** (a hearty stew), and **mantecadas** (sweet pastries) are must-try delicacies that visitors can savor at charming cafes and local restaurants. The **Museo del Chocolate** provides insight into the history of chocolate-making in the region, offering tastings and exhibits (Wanderlog, n.d.).

### 3.2 Free Walking Tours

To explore Astorga's rich history and cultural offerings, visitors can join a **Free Walking Tour**, led by knowledgeable local guides. These tours typically cover the town's key attractions and provide valuable insights into its historical significance and architectural marvels. While the tours are free, donations to the guides are appreciated (World Free Tours, n.d.).

## 4. Outdoor Activities

Astorga's scenic surroundings provide ample opportunities for outdoor enthusiasts to explore the beautiful landscapes of the region.

### 4.1 Camino de Santiago

As a popular stop on the **Camino de Santiago**, Astorga attracts pilgrims and hikers from around the world. The Camino offers a unique chance to experience the stunning Spanish countryside while engaging in a centuries-old tradition of pilgrimage. Visitors can choose to walk a portion of the route, immersing themselves in the natural beauty and spiritual significance of this historic path (Eye on Spain, n.d.).

### 4.2 Hiking and Nature Trails

In addition to the Camino, Astorga is surrounded by various hiking trails that lead to picturesque villages and breathtaking views. The **Gold Route** is a notable path that explores ancient Roman mines, providing a glimpse into the region's mining history while allowing hikers to enjoy the tranquility of the countryside (Eye on Spain, n.d.).

## 5. Key Attractions and Events

### 5.1 Roman Museum of Astorga

The **Roman Museum** is an essential stop for anyone interested in Astorga's rich Roman heritage. The museum features a variety of artifacts, including mosaics, sculptures, and coins, that bring the ancient past to life (Tourist Places Guide, n.d.).

### 5.2 Cultural Events

Astorga hosts various cultural events throughout the year, including traditional festivals and markets. The vibrant atmosphere during these events allows visitors to engage with local customs and traditions, enhancing their overall experience in the town (Eye on Spain, n.d.).

## 6. Accessibility

Astorga is easily accessible for travelers, with the nearest airport located in **León (LEN)**. Additionally, there are scenic bus and train routes available from **Santiago de Compostela (SCQ)** and **Santander (SDR)**, making it convenient for visitors to reach this charming town (Turispanish, n.d.).

## 7. Conclusion

Astorga, Spain, is a captivating destination that offers a rich tapestry of history, culture, and natural beauty. With its well-preserved Roman ruins, stunning architecture, vibrant culinary scene, and outdoor activities, visitors are sure to have an unforgettable experience. Whether exploring the iconic Episcopal Palace, savoring local delicacies, or walking the historic Camino de Santiago, Astorga promises a unique blend of ancient and modern experiences that cater to a wide range of interests.

As you plan your visit, consider taking advantage of the local walking tours and engaging with the community to fully immerse yourself in the culture and history that Astorga has to offer.

## References

- Eye on Spain. (n.d.). Astorga - A must visit. Retrieved from https://www.eyeonspain.com/blogs/bestofspain/23350/astorga---a-must-visit.aspx
- Lonely Planet. (n.d.). Activities beyond the trails. Retrieved from https://www.lonelyplanet.com/articles/camino-de-santiago-activities-beyond-the-trails
- Turispanish. (n.d.). Tourism in Astorga, Spain. Retrieved from https://turispanish.com/tourism/astorga-spain-tourism/
- Wanderlog. (n.d.). Top things to do and attractions in Astorga. Retrieved from https://wanderlog.com/list/geoCategory/1589429/top-things-to-do-and-attractions-in-astorga
- World Free Tours. (n.d.). Explore Astorga for free. Retrieved from https://worldfreetours.com/blog/astorga/explore-astorga-for-free/
- Tourist Places Guide. (n.d.). Top 10 must-visit tourist places in Astorga. Retrieved from https://touristplaces.guide/top-10-must-visit-tourist-places-in-astorga/

# Re-writing using chains

In [23]:
from prompts import (
    RESEARCH_REPORT_PROMPT_TEMPLATE
)
from chain_1_1 import assistant_instructions_chain
from chain_2_1 import web_searches_chain
from chain_3_1 import search_result_urls_chain, web_search
from chain_4_1 import search_result_text_and_summary_chain, web_scrape
 
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
 
search_and_summarization_chain = (
    search_result_urls_chain 
    | search_result_text_and_summary_chain.map() # parallelize for each url
    | RunnableLambda(lambda x: 
        {
            'summary': '\n'.join([i['summary'] for i in x]), 
            'user_question': x[0]['user_question'] if len(x) > 0 else ''
        })
)

web_research_chain = (
    assistant_instructions_chain 
    | web_searches_chain 
    | search_and_summarization_chain.map() # parallelize for each web search
    | RunnableLambda(lambda x:
       {
           'research_summary': '\n\n'.join([i['summary'] for i in x]),
           'user_question': x[0]['user_question'] if len(x) > 0 else ''
        })
    | RESEARCH_REPORT_PROMPT_TEMPLATE | get_llm() | StrOutputParser()
)

In [24]:
question = 'What can I see and do in the Spanish town of Astorga?'
 
web_research_report = web_research_chain.invoke(question)
print(web_research_report)

name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined
name 'requests' is not defined


# Exploring Astorga, Spain: A Guide to Attractions and Activities

Astorga, a charming town located in the province of León in northwestern Spain, serves as a gateway to the rich history and culture of the region. While the sources reviewed presented technical errors and did not provide specific information about attractions or activities in Astorga, this report aims to synthesize known facts about the town based on general knowledge and historical context. This structured report will explore the various attractions, cultural activities, and historical significance of Astorga, offering a comprehensive overview for those looking to visit this picturesque town.

## Historical Background

Astorga has a deep-rooted history that dates back to Roman times when it was known as Asturica Augusta. The town was strategically located along the Roman road that connected the north of Spain with the southern regions, making it an essential stop for travelers and traders. The remnants of Roman archite

In [27]:
display(Markdown(web_research_report))

# Exploring Astorga, Spain: A Guide to Attractions and Activities

Astorga, a charming town located in the province of León in northwestern Spain, serves as a gateway to the rich history and culture of the region. While the sources reviewed presented technical errors and did not provide specific information about attractions or activities in Astorga, this report aims to synthesize known facts about the town based on general knowledge and historical context. This structured report will explore the various attractions, cultural activities, and historical significance of Astorga, offering a comprehensive overview for those looking to visit this picturesque town.

## Historical Background

Astorga has a deep-rooted history that dates back to Roman times when it was known as Asturica Augusta. The town was strategically located along the Roman road that connected the north of Spain with the southern regions, making it an essential stop for travelers and traders. The remnants of Roman architecture, including a well-preserved aqueduct and ruins of a Roman fort, still highlight the town's historical significance.

Throughout the Middle Ages, Astorga flourished as a crucial center for the pilgrimage routes to Santiago de Compostela, further enhancing its cultural and economic importance. The town was granted the title of a city in the 11th century and has continued to develop its identity through various historical epochs.

## Architectural Highlights

Astorga is renowned for its eclectic architecture, which reflects its diverse historical influences. Below are some of the notable architectural landmarks:

### 1. **Astorga Cathedral**
The Astorga Cathedral, officially known as the Cathedral of Santa María, is a stunning example of Gothic architecture, although it also incorporates elements of Romanesque and Baroque styles. The construction began in the 16th century, and the cathedral was completed in the 18th century. Visitors can admire its intricate façade, beautiful stained-glass windows, and the impressive altarpiece within.

### 2. **Gaudí's Palace (Palacio Episcopal)**
Designed by the renowned Catalan architect Antoni Gaudí, the Episcopal Palace of Astorga is a striking structure that combines neo-Gothic and modernist styles. Built between 1889 and 1915, the palace features whimsical turrets and ornate detailing that reflect Gaudí's unique architectural vision. Today, it serves as a museum showcasing the history of the episcopal see and the work of Gaudí.

### 3. **Roman Walls**
The remnants of the Roman walls that once fortified Astorga are still visible today. Dating back to the 1st century AD, these walls provide insight into the town's ancient past. Walking along the walls offers a unique perspective of the town and its surroundings.

## Cultural Activities

Astorga is a hub of cultural activities that reflect its rich history and vibrant community life. Visitors can engage in various experiences that highlight the town's heritage.

### 1. **Local Festivals**
Astorga hosts several annual festivals that celebrate its cultural heritage. One of the most notable is the Feast of St. James (Fiestas de Santiago), celebrated in late July. This festival includes religious processions, traditional music, dancing, and culinary delights, providing visitors with an immersive cultural experience.

### 2. **Culinary Delights**
The gastronomy of Astorga is a significant aspect of its culture. The town is famous for its "cocido maragato," a hearty stew made with a variety of meats and vegetables. Additionally, Astorga is known for its artisan chocolates, which are highly regarded throughout Spain. Visitors can explore local eateries and chocolatiers to taste these delightful treats.

### 3. **Museums and Exhibits**
Astorga is home to several museums that showcase its history and culture. The Museo de los Caminos (Museum of the Ways) focuses on the town's role in the pilgrimage routes, while the Museo del Chocolate (Chocolate Museum) delves into the history and production of chocolate in the region. These museums provide educational and engaging experiences for visitors of all ages.

## Outdoor Activities

Astorga is surrounded by natural beauty, making it an excellent destination for outdoor enthusiasts. The landscape offers opportunities for hiking, cycling, and exploring the picturesque countryside.

### 1. **Walking and Cycling Routes**
The town is situated along the Camino de Santiago, making it a popular stop for pilgrims on their journey. The surrounding areas offer numerous walking and cycling routes that allow visitors to explore the scenic landscapes of the León province. These routes vary in difficulty, catering to both casual walkers and experienced hikers.

### 2. **Nearby Natural Attractions**
Astorga is located near the stunning landscapes of the Sierra de Gredos and the Picos de Europa National Park. These areas are known for their dramatic mountain scenery, diverse wildlife, and abundant recreational activities. Visitors can take day trips to these natural parks to enjoy hiking, birdwatching, and photography.

## Conclusion

Astorga, with its rich historical background, stunning architecture, vibrant cultural scene, and beautiful natural surroundings, offers a wide range of attractions and activities for visitors. The town's unique blend of history and modernity provides a captivating experience for anyone looking to explore the heart of Spain. While the sources reviewed did not provide specific information, the general knowledge about Astorga allows for a well-rounded understanding of what the town has to offer.

For those planning to visit Astorga, it is advisable to immerse oneself in the local culture, taste the delicious cuisine, and explore the historical sites that define this remarkable town. Astorga is not only a stop along the Camino de Santiago but also a destination in its own right, beckoning travelers to discover its secrets and charms.

## References

1. https://wanderlog.com/list/geoCategory/1589429/top-things-to-do-and-attractions-in-astorga
2. https://touristplaces.guide/top-10-must-visit-tourist-places-in-astorga/
3. https://www.eyeonspain.com/blogs/bestofspain/23350/astorga---a-must-visit.aspx
4. https://turispanish.com/tourism/astorga-spain-tourism/

(Note: The URLs referenced do not contain specific information due to technical issues but have been included for completeness based on the user's request.)